In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
test = pd.concat([test,postulaciones])
test = test.drop_duplicates(['idaviso','idpostulante'])
postulaciones = test[100000:].drop('id', axis=1)
test = test[:100000].drop('id', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1112248724,NjlD,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
1,1112248724,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
2,1112304156,MV6eM8N,1.0,Full-time,Junior,Comunicacion,RH Talentum,34,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
3,1112445502,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,34,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
4,1112254843,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,64,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC


In [5]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,739260,6M9ZQR,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1976-03-29,FEM
1,739260,6v1xdL,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1987-06-27,MASC
2,739260,ezRKm9,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1982-02-17,FEM
3,758580,1Q35ej,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1949-12-24,MASC
4,758580,EAN4J6,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1986-05-27,FEM


In [6]:
def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        #date_new = int(date_new)
        return date_new
    except ValueError:
        return -1

In [7]:
fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [8]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    print(col)
    postulaciones[col] = postulaciones[col].fillna('na')
    test[col] = test[col].fillna('na')
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1,4,120,2428,33,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1
1,1,4,120,2428,33,0.0,0.0,0.0,0.0,0.0,3.0,2.0,27.0,2
2,1,2,36,3236,34,0.0,0.0,0.0,0.0,0.0,3.0,2.0,27.0,2
3,1,4,36,2228,34,0.0,0.0,0.0,0.0,0.0,3.0,2.0,27.0,2
4,1,4,64,154,64,0.0,0.0,0.0,0.0,0.0,3.0,2.0,27.0,2


In [10]:
postulaciones.sample(50)

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
12253124,2,2,174,505,79,0.0,0.0,0.0,0.0,0.0,0.0,2.0,22.0,1
11183425,1,3,47,2747,41,0.0,0.0,0.0,0.0,0.0,0.0,2.0,32.0,1
11690016,1,4,115,649,70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1
369151,1,4,30,965,7,0.0,0.0,0.0,0.0,3.0,0.0,3.0,25.0,1
8321601,1,2,27,2006,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,2
204903,1,4,6,3669,46,0.0,0.0,0.0,0.0,0.0,3.0,0.0,25.0,1
147584,2,3,26,1342,28,0.0,0.0,0.0,0.0,3.0,0.0,2.0,21.0,1
5109068,2,4,2,417,43,0.0,0.0,0.0,0.0,3.0,0.0,2.0,22.0,2
11626764,1,4,30,1816,30,0.0,2.0,0.0,0.0,0.0,0.0,3.0,36.0,1
2066168,2,2,42,1628,12,0.0,0.0,0.0,0.0,3.0,0.0,2.0,23.0,1


In [11]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [12]:
aviso = None
postulantes = None
postulaciones.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True)
clf.fit(postulaciones,resultado)
y_pred=clf.predict(test)



In [ ]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = y_pred
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

In [ ]:
y_pred[y_pred==1].shape